In [1]:
import { requireCytoscape, requireCarbon } from "./lib/draw";

requireCarbon();
requireCytoscape();

# Meta-Programming

In [2]:
import { draw, treeLayout } from "./lib/draw";
import * as T from "./lib/lambdats/token";
import * as E from "./lib/lambdats/expr";
import * as I from "./lib/lambdats/substInterp";
import * as Transpiler from "./lib/lambdats/transpile";
import * as Parser from "./lib/lambdats/parser";

function drawProg(prog: string|E.Expr): void {
    if (typeof prog === 'string') {
        draw(E.cytoscapify(Parser.parse(prog)), 800, 350, treeLayout);
    } else {
        draw(E.cytoscapify(prog), 800, 350, treeLayout);
    }
}

## Where Were We?

1. Language primitives (i.e., building blocks of languages)
2. Language paradigms (i.e., combinations of language primitives)
3. **Building a language** (i.e., designing your own language)
    * Last time: compilers and transpilers
    * This time: **meta-programming** at run-time

## Meta-Programming = The "Final" Programming Paradigm

1. Meta-programming: a program that modifies/generates programs.
2. We already saw 1 "meta-program", a compiler/transpiler.
3. Today, we will look at eval/apply form of meta-programming, or, the idea that "code is data and data is code".
4. This let's us change our code as we run it. This is different from **macros**, or static/compile-time meta-programming.

## Eval/Apply: Code is Data, Data is Code

1. Code = Expression, i.e., an AST that still has computation left to be run.
2. Data = Value, i.e., a subset of expressions that has no computation left to be run. Recall that a value can contain anonymous functions, which when called with an argument, can run computation.
3. **Eval** will take some data and turn it into code. **Apply** will take some code and use it to produce data.

In [3]:
import * as ts from "typescript";

### 1. Input TypeScript Program

In [4]:
function f(x: number): number { // code
    return x + 1;
}

f(1)

2


#### Corresponding LambdaTS2 program
```ts
let f = (λx => x + 1) in 
f(1)
```

### 2. Input TypeScript Program as a TypeScript String

In [5]:
// This string just happens to contain a valid TypeScript Program, data
`
function f(x: number): number { // code
    return x + 1;
}

f(1)
`


function f(x: number): number { // code
    return x + 1;
}

f(1)



In [6]:
// We can also convert the function directly to a string, albeit in a lossy manner
const source: string = f.toString() + "\n\nf(1);";
source

function f(x) {
    return x + 1;
}

f(1);


### 3. TypeScript Program that Parses TypeScript Strings 

In [7]:
// ts.createSourceFile is a Lexer+Parser
const sourceFile: ts.SourceFile = ts.createSourceFile('tmp.ts', source, ts.ScriptTarget.ES2015, true, ts.ScriptKind.TS);

// Now that we have an AST, we can write an interpreter / compiler
sourceFile

<ref *1> SourceFileObject {
  pos: 0,
  end: 42,
  flags: 0,
  modifierFlagsCache: 0,
  transformFlags: 64,
  parent: undefined,
  kind: 300,
  statements: [
    NodeObject {
      pos: 0,
      end: 35,
      flags: 0,
      modifierFlagsCache: 0,
      transformFlags: 2097216,
      parent: [Circular *1],
      kind: 254,
      decorators: undefined,
      modifiers: undefined,
      symbol: undefined,
      localSymbol: undefined,
      locals: undefined,
      nextContainer: undefined,
      name: [IdentifierObject],
      typeParameters: undefined,
      parameters: [Array],
      type: undefined,
      body: [NodeObject],
      asteriskToken: undefined
    },
    NodeObject {
      pos: 35,
      end: 42,
      flags: 0,
      modifierFlagsCache: 0,
      transformFlags: 0,
      parent: [Circular *1],
      kind: 236,
      expression: [NodeObject]
    },
    pos: 0,
    end: 42,
    hasTrailingComma: false,
    transformFlags: 64
  ],
  endOfFileToken: TokenObject {
    pos: 42

#### Corresponding LambdaTS2 AST

In [8]:
const inputString = "let f = λx => x + 1 in f(1)";  // source
const outputAST = Parser.parse(inputString);        // sourceFile, Parser.parse is ts.createSourceFile
drawProg(outputAST)

#### TypeScript AST

If you're interested in exploring the AST for TypeScript, this is a good link [TypeScript AST Viewer](https://ts-ast-viewer.com/#).

### 4. TypeScript Program that Converts TypeScript AST Back into String

In [9]:
const printer = ts.createPrinter({ newLine: ts.NewLineKind.LineFeed });
printer.printFile(sourceFile)

function f(x) {
    return x + 1;
}
f(1);



#### Corresponding LambdaTS2 Printer

In [10]:
drawProg(outputAST)
E.exprToString(outputAST)

let f = (λx=>(x+1)) in f(1)


### 5. TypeScript Program that Converts TypeScript String to JavaScript String

In [11]:
// sourceFile was a TypeScript AST
// It's kind of inefficient to convert the AST back into a string
const jsSource = ts.transpile(printer.printFile(sourceFile));
jsSource

function f(x) {
    return x + 1;
}
f(1);



#### Same thing in LambdaTS World

In [12]:
const lambdaTsAST = Transpiler.transpile(outputAST); // LambdaTS2 -> LambdaTS
const lambdaTsSource = E.exprToString(lambdaTsAST);
console.log("original", "let f = (λx=>(x+1)) in f(1)");
console.log("transpiled", lambdaTsSource);
drawProg(lambdaTsAST);

original let f = (λx=>(x+1)) in f(1)
transpiled (λf=>f(1))((λx=>(x+1)))


### 6. JavaScript Function that Converts JavaScript String to JavaScript Function, or Eval

In [13]:
const f = (x) => x + 1;
`return ${f(1)}` // Takes arbitrary TypeScript code, runs it, and converts it into a string

return 2


In [14]:
console.log("original\n", jsSource);
console.log("\n\nstring interpolation\n");
`return ${jsSource}` // Takes arbitrary TypeScript code, runs it, and converts it into a string

original
 function f(x) {
    return x + 1;
}
f(1);



string interpolation

return function f(x) {
    return x + 1;
}
f(1);



In [15]:
const g = Function(`return ${jsSource}`); // "eval", i.e., "paste" into Jupyter repl
console.log("g is function", g);
console.log("\n\nremember, we could convert a function to a string", g.toString());

g is function [Function: anonymous]


remember, we could convert a function to a string function anonymous(
) {
return function f(x) {
    return x + 1;
}
f(1);

}


In [16]:
const h = g(); // h is a TypeScript Function that is the result of string manipulation
console.log("source", h.toString());
h(1)

source function f(x) {
    return x + 1;
}
2


#### Eval for LambdaTS2?

- We need to write a parser and interpreter in LambdaTS2 in LambdaTS2

In [17]:
console.log(lambdaTsSource);
const thereAndBackAST = Parser.parse(lambdaTsSource);
drawProg(thereAndBackAST);

(λf=>f(1))((λx=>(x+1)))


### 7. JavaScript Function is TypeScript Function

In [18]:
console.log("Original");
console.log(f.toString());
console.log("From AST to String and Back");
console.log(h.toString());

Original
(x) => x + 1
From AST to String and Back
function f(x) {
    return x + 1;
}


In [19]:
[f(3), h(3)] // Using TypeScript interpreter, i.e., apply

[ 4, 4 ]


In [20]:
[f(5), h(5)] // Uses TypeScript interpreter, i.e., apply

[ 6, 6 ]


#### Corresponding Paradigm in LambdaTS2?

- Again, we would need to write an interpreter for LambdaTS2 in LambdaTS2.
- Here, we're using the interpreter for LambdaTS (remember we transpiled) that we wrote in TypeScript.

In [21]:
console.log(lambdaTsSource);
drawProg(I.interpret(lambdaTsAST))

(λf=>f(1))((λx=>(x+1)))


In [22]:
console.log(lambdaTsSource);
drawProg(I.interpret(thereAndBackAST));

(λf=>f(1))((λx=>(x+1)))


## Application: Code Instrumentation

- Problem: suppose you want to debug a function by printing out all of its arguments. 
- Solution: write a meta-program that instruments your code to print out all of its arguments.

These ideas might appear in:
1. Code editor
2. Debugger
3. Logging system

In [23]:
const printFunctionArguments = <T extends ts.Node>(context: ts.TransformationContext) => (rootNode: T) => {
    function traceParam(param: ts.ParameterDeclaration): ts.ArrayLiteralExpression {
        const paramStr = ts.factory.createStringLiteral(param.name.getText());
        const paramName = ts.factory.createIdentifier(param.name.getText());
        return ts.factory.createArrayLiteralExpression([paramStr, paramName]);
    }
    
    function visit(node: ts.Node): ts.Node {
        switch (node.kind) {
            case ts.SyntaxKind.FunctionDeclaration: {
                const func = node as ts.FunctionDeclaration;
                const body2 = ts.visitEachChild(func.body, visit, context); // Take original body
                const params = ts.factory.createArrayLiteralExpression(func.parameters.map(traceParam));
                const functionName = ts.factory.createIdentifier("console.log");
                const callLog = ts.factory.createCallExpression(functionName, undefined, [params]);
                const prelude: ts.Statement[] = [ts.factory.createExpressionStatement(callLog)];
                const body3 = ts.factory.createBlock(prelude.concat(body2.statements), true); // Convert to this
                return ts.factory.createFunctionDeclaration(func.decorators, func.modifiers, func.asteriskToken, func.name, func.typeParameters, func.parameters, func.type, body3);
            }
            default: {
                return ts.visitEachChild(node, visit, context);
            }
        }
    };
    return ts.visitNode(rootNode, visit);
};

In [24]:
function tsAndBack(f: any, transformer: any, verbose=false): any { // Hard to write down types with meta-programming
    // 1. f is input typescript code
    
    // 2. Input TypeScript Program as a TypeScript String
    const source = f.toString();
    
    // 3. TypeScript Program that Parses TypeScript Strings 
    const sourceFile: ts.SourceFile = ts.createSourceFile('tmp.ts', source, ts.ScriptTarget.ES2015, true, ts.ScriptKind.TS);
    
    // This part is a compiler/transpiler!!
    const result: ts.TransformationResult<ts.SourceFile> = ts.transform<ts.SourceFile>(sourceFile, [transformer]);
    const transformedSourceFile: ts.SourceFile = result.transformed[0];
    
    // 4. TypeScript Program that Converts TypeScript AST Back into String
    const printer = ts.createPrinter({ newLine: ts.NewLineKind.LineFeed });
    const transformedString = printer.printFile(transformedSourceFile);
    if (verbose) {
        console.log(transformedString);
    }
    
    // 5. TypeScript Program that Converts TypeScript String to JavaScript String
    const jsSource = ts.transpile(printer.printFile(transformedSourceFile));
    
    // 6. JavaScript Function that Converts JavaScript String to JavaScript Function
    const g = Function(`return ${jsSource}`);
    
    // 7. JavaScript Function is TypeScript Function
    return g();
}


### Example 1

In [25]:
function g(x: number, z: number): number {
    return x + z;
}

g(3, 1);

4


In [26]:
const gInstrumented = tsAndBack(g, printFunctionArguments, true);
gInstrumented(3, 1)

function g(x, z) {
    console.log([["x", x], ["z", z]]);
    return x + z;
}

[ [ 'x', 3 ], [ 'z', 1 ] ]
4


### Example 2

In [27]:
const g = (x: number, y: any): number => x;

const gInstrumented = tsAndBack(g, printFunctionArguments, true);

(x, y) => x;



In [28]:
gInstrumented(3, "hi")

3


### Hmmm ... what went wrong? 

The function declaration AST and the anonymous function AST in typescript are different.

In [29]:
const printFunctionArguments2 = <T extends ts.Node>(context: ts.TransformationContext) => (rootNode: T) => {
    function traceParam(param: ts.ParameterDeclaration): ts.ArrayLiteralExpression {
        const paramStr = ts.factory.createStringLiteral(param.name.getText());
        const paramName = ts.factory.createIdentifier(param.name.getText());
        return ts.factory.createArrayLiteralExpression([paramStr, paramName]);
    }
    
    function visit(node: ts.Node): ts.Node {
        switch (node.kind) {
            case ts.SyntaxKind.ArrowFunction: {
                const arr = node as ts.ArrowFunction;
                const body2 = ts.visitEachChild(arr.body, visit, context);
                const params = ts.factory.createArrayLiteralExpression(arr.parameters.map(traceParam));
                const functionName = ts.factory.createIdentifier("console.log");
                const instrumented = ts.factory.createCallExpression(functionName, undefined, [params])
                const tuple = ts.factory.createArrayLiteralExpression([body2 as ts.Expression].concat(instrumented));
                const body3 = ts.factory.createElementAccessExpression(tuple, 0);
                return ts.factory.createArrowFunction(arr.modifiers, arr.typeParameters, arr.parameters, arr.type, arr.equalsGreaterThanToken, body3);
            }
            case ts.SyntaxKind.FunctionDeclaration: {
                const func = node as ts.FunctionDeclaration;
                const body2 = ts.visitEachChild(func.body, visit, context);
                const params = ts.factory.createArrayLiteralExpression(func.parameters.map(traceParam));
                const functionName = ts.factory.createIdentifier("console.log");
                const prelude: ts.Statement[] = [ts.factory.createExpressionStatement(ts.factory.createCallExpression(functionName, undefined, [params]))];
                const body3 = ts.factory.createBlock(prelude.concat(body2.statements), true);
                return ts.factory.createFunctionDeclaration(func.decorators, func.modifiers, func.asteriskToken, func.name, func.typeParameters, func.parameters, func.type, body3);
            }
            default: {
                return ts.visitEachChild(node, visit, context);
            }
        }
    };
    return ts.visitNode(rootNode, visit);
};

### Example 2 Again

In [30]:
const g = (x: number, y: any): number => x;

const gInstrumented = tsAndBack(g, printFunctionArguments2, true);

(x, y) => [x, console.log([["x", x], ["y", y]])][0];



In [31]:
gInstrumented(3, "hi")

[ [ 'x', 3 ], [ 'y', 'hi' ] ]
3


## Application: Making Your Code Lazy

In [32]:
const lazify = <T extends ts.Node>(context: ts.TransformationContext) => (rootNode: T) => {    
    function visit(node: ts.Node): ts.Node {
        switch (node.kind) {
            // Lazy Transformation
            case ts.SyntaxKind.Identifier: {
                const ident = node as ts.Identifier;
                const functionName = ts.factory.createIdentifier(ident.getFullText());
                return ts.factory.createCallExpression(
                    functionName,
                    undefined, /*typeArgs*/ 
                    [ts.factory.createNumericLiteral(42)]
                );
            }
            case ts.SyntaxKind.CallExpression: {
                const call = node as ts.CallExpression;
                const thunks = call.arguments.map((arg) => ts.factory.createArrowFunction(
                    undefined, /* modifiers: readonly ts.Modifier[] */
                    undefined, /* typeParameters: readonly ts.TypeParameterDeclaration[] */
                    [],        /* parameters: readonly ts.ParameterDeclaration[] */
                    undefined, /* type: ts.TypeNode */
                    ts.factory.createToken(ts.SyntaxKind.EqualsGreaterThanToken),
                    ts.visitNode(arg, visit)
                ));
                return ts.factory.createCallExpression(ts.visitNode(call.expression, visit), undefined, thunks);
                return node;
            }
            // Boiler plate
            case ts.SyntaxKind.ArrowFunction: { // Do not recurse on function parameters
                const arr = node as ts.ArrowFunction;
                const body2 = ts.visitNode(arr.body, visit);
                return ts.factory.createArrowFunction(arr.modifiers, arr.typeParameters, arr.parameters, arr.type, arr.equalsGreaterThanToken, body2);
            }
            case ts.SyntaxKind.FunctionDeclaration: { // Do not recurse on function parameters
                const func = node as ts.FunctionDeclaration;
                const body2 = ts.visitNode(func.body, visit);
                return ts.factory.createFunctionDeclaration(func.decorators, func.modifiers, func.asteriskToken, func.name, func.typeParameters, func.parameters, func.type, body2);
            }
            default: {
                return ts.visitEachChild(node, visit, context);
            }
        }
    };
    return ts.visitNode(rootNode, visit);
};

In [33]:
function h(x) {
    return ((x, y) => 1 + x)(1, ((z) => {console.log("Printing", x)})());
}

h(10)

Printing 10
2


In [34]:
const hLazy = tsAndBack(h, lazify, true);
hLazy(10)

function h(x) {
    return ((x, y) => 1 +  x(42))(() => 1, () => ((z) => {  console(42).log(42)(() => "Printing", () =>  x(42)); })());
}

2


The two examples that we've seen are examples of **run-time** meta-programming.

Languages that have this include:
- TypeScript: what we've just seen
- Python
- Lisp/Scheme (**homoiconic**, i.e., the syntax and the code are the same representation)

## Macros are Static Meta-programming

1. The idea of writing code to modify code at compile-time is called a **macro**.
2. Macros are an example of meta-programming.
3. Langauges with macros:
    - C (`#include`)
    - Scala (similar to TypeScript)

## Summary

1. A meta-program is a program that takes programs as inputs and produces them as outputs.
2. We saw the compiler API for TypeScript, which allows us to write TypeScript meta-programs.
3. More generally, these compile-time meta-programs that operate on themselves are called macros.
4. Meta-programming is the "final programming paradigm" that ties together the idea that code is data and data is code through eval/apply.